In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [2]:
# Define the data directories
train_dir = '../Data/Train/'
valid_dir = '../Data/Test/'

# Set the image size and batch size
set_img_size = (255, 255)
set_batch_size = 5


In [3]:
from keras.preprocessing.image import ImageDataGenerator

# Define the data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=set_img_size,
    batch_size=set_batch_size,
    class_mode='categorical',
    shuffle=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=set_img_size,
    batch_size=set_batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 1560 images belonging to 4 classes.
Found 180 images belonging to 4 classes.


In [4]:
print(train_generator.class_indices)

{'Green_Bad': 0, 'Green_Good': 1, 'Yellow_Bad': 2, 'Yellow_Good': 3}


In [6]:
from keras import Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(
        set_img_size[0], set_img_size[1], 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])


def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 * (10 - epoch))


lr_schedule = LearningRateScheduler(scheduler)

model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3
)

checkpoint_path = "../Save/Checkpoint/cp-{epoch:04d}.h5"
checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 253, 253, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 126, 126, 32)     0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 126, 126, 32)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 62, 62, 64)       

In [ ]:
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=valid_generator,
    callbacks=[early_stopping, lr_schedule, checkpoint]
)

In [7]:
from keras.models import load_model
model = load_model('../Save/mango_tensorflow_96_0.12.h5')
set_img_size = (255, 255)


In [10]:
from keras.applications.vgg16 import preprocess_input

image_path = '../Data/Test/Yellow_Good/49.jpg'
image = tf.io.read_file(image_path)
image = tf.image.decode_image(image, channels=3)
image = tf.expand_dims(image, axis=0)
image = tf.image.resize(image, size=(set_img_size[0], set_img_size[1]))
x = preprocess_input(image)

preds = model.predict(x)

class_idx = np.argmax(preds[0])
class_label = train_generator.class_indices
for k, v in class_label.items():
    if v == class_idx:
        print(f"Predicted class: {k}")
        break

for i, prob in enumerate(preds[0]):
    print(f'Class {i}: {prob}')


1/1 [==============================] - 0s 230ms/step
Predicted class: Yellow_Good
Class 0: 0.0
Class 1: 0.0
Class 2: 1.426343567915691e-22
Class 3: 1.0
